# EJERCICIOS

In [13]:
import warnings
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier,StackingClassifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

In [14]:
warnings.filterwarnings("ignore", category=FutureWarning)

## EJERCICIO 1

Si has entrenado cinco modelos diferentes en el mismo conjunto de entrenamiento exacto y todos consiguen una precisión del 95%, ¿hay alguna posibilidad de que puedas combinar estos modelos para obtener mejores resultados? 

Si la respuesta es sí, ¿cómo? Si la respuesta es no, ¿por qué?

Si, se pueden combinar. Hay varios metodos de combinación, el de votación, el de la media, el boosting y el bagging. Habría que elegir el adecuado. Al combinar modelos, podemos mejorar la precisión al basarse en los diferentes modelos en vez de en uno. Además, si los modelos estan entrenados con diferentes datos, esa diversidad puede ser clave para mejorar la precisión, ya que tendrá más valores de los que aprender. 

## EJERCICIO 2

Carga el conjunto de datos MNIST y divídelo en un conjunto de entrenamiento, un conjunto de validación y un conjunto de prueba (por ejemplo, utiliza 50.000 instancias para entrenamiento, 10.000 para validación y 10.000 para pruebas). 

Después, entrena varios clasificadores diferentes (uno de ellos que sea un árbol de decisión). 

A continuación, intenta combinarlos en un ensamble que supere en rendimiento a cada clasificador individual del conjunto de validación, utilizando hard voting. 

Una vez que hayas encontrado uno, pruébalo en el conjunto de pruebas.

In [15]:
mnist = fetch_openml('mnist_784', as_frame=False, parser="auto")
X_mnist = mnist.data
y_mnist =  mnist.target

# 50.000 para entrenamiento
X_train, y_train = X_mnist[:50_000], y_mnist[:50_000]
# Las siguientes 10.000 para validación
X_valid, y_valid = X_mnist[50_000:60_000], y_mnist[50_000:60_000]

# Y las siguientes 10.000 para pruebas
X_test, y_test = X_mnist[60_000:], y_mnist[60_000:]

In [17]:
# usar los clasifiers
arbol_decision = DecisionTreeClassifier(random_state=42)
random_forest = RandomForestClassifier(random_state=42)
svm = SVC(random_state=42)
kvecinos = KNeighborsClassifier()

# Agrupar todos los estimadores en una lista para facilitar el procesamiento
estimadores = [arbol_decision, random_forest, svm, kvecinos]

# Entrenar cada estimador individualmente
for estimator in estimadores:
    print("Entrenando", estimator.__class__.__name__)
    estimator.fit(X_train, y_train)  # Entrenar el modelo con los datos de entrenamiento

# Obtener las puntuaciones de validación para cada estimador
# Calcula y almacena la precisión de cada modelo en los datos de validación
scores = [estimator.score(X_valid, y_valid) for estimator in estimadores]

# Asignar nombres a los estimadores para usarlos en el VotingClassifier
named_estimators = [
    ("Árbol de decisión", arbol_decision),
    ("Random Forest", random_forest),
    ("SVM", svm),
    ("KVecinos", kvecinos)
]

# Crear un clasificador de votación que combina las predicciones de múltiples modelos
voting_clf = VotingClassifier(estimators=named_estimators)

# Entrenar el VotingClassifier con los datos de entrenamiento
voting_clf.fit(X_train, y_train)

# Evaluar el VotingClassifier en los datos de validación
error_val = voting_clf.score(X_valid, y_valid)
print("Precisión del conjunto de validación:", error_val)

# Evaluar el VotingClassifier en los datos de prueba
error_test = voting_clf.score(X_test, y_test)
print("Precisión del conjunto de prueba:", error_test)

Entrenando DecisionTreeClassifier
Entrenando RandomForestClassifier
Entrenando SVC
Entrenando KNeighborsClassifier
Precisión del conjunto de validación: 0.9788
Precisión del conjunto de prueba: 0.974


## EJERCICIO 3

Ejecuta los clasificadores individuales del ejercicio anterior para hacer predicciones en el conjunto de entrenamiento y crea un nuevo conjunto de entrenamiento con las predicciones resultantes: cada instancia de entrenamiento es un vector que contiene el conjunto de predicciones de todos tus clasificadores para una imagen y el objetivo es la clase de la imagen. Entrena un clasificador (RandomForestClassifier) en este nuevo conjunto de entrenamiento. 

Acabas de entrenar un blender y, junto a los clasificadores, forma un ensamble de stacking.

Ahora, evalúa el ensamble en el conjunto de prueba. 

¿Cómo es en comparación con el clasificador de votación que has entrenado antes?

Haz lo mismo usando StackingClassifier

In [ ]:
# Cuidado con esto, tarda 35 minutos en ejecutar
# Generar predicciones del conjunto de entrenamiento con cada clasificador
predicciones_entrenamiento = []
for estimator in estimadores:
    print(f"Generando predicciones con {estimator.__class__.__name__}")
    predicciones = estimator.predict(X_train)
    predicciones_entrenamiento.append(predicciones)

# Crear el nuevo conjunto de entrenamiento con las predicciones de los clasificadores
X_train_blender = np.array(predicciones_entrenamiento).T  # ajustar la forma
y_train_blender = y_train  # El target es el mismo

# Entrenar el blender con RandomForestClassifier
blender = RandomForestClassifier(random_state=42)
blender.fit(X_train_blender, y_train_blender)

# Evaluar el blender en el conjunto de prueba
predicciones_prueba = []
for estimator in estimadores:
    predicciones = estimator.predict(X_test)
    predicciones_prueba.append(predicciones)

X_test_blender = np.array(predicciones_prueba).T  # ajustar la forma
error_test_blender = blender.score(X_test_blender, y_test)
print("Precisión del ensamble (blender):", error_test_blender)

# Comparación con VotingClassifier
print("Precisión del conjunto de prueba (VotingClassifier):", error_test)

# Usar StackingClassifier para verificar el rendimiento
stacking_clf = StackingClassifier(
    estimators=named_estimators,
    final_estimator=RandomForestClassifier(random_state=42)
)
stacking_clf.fit(X_train, y_train)
error_test_stacking = stacking_clf.score(X_test, y_test)
print("Precisión del conjunto de prueba (StackingClassifier):", error_test_stacking)


Generando predicciones con DecisionTreeClassifier
Generando predicciones con RandomForestClassifier
Generando predicciones con SVC
Generando predicciones con KNeighborsClassifier
Precisión del ensamble (blender): 0.9288
Precisión del conjunto de prueba (VotingClassifier): 0.974
Precisión del conjunto de prueba (StackingClassifier): 0.9806
